# Demo

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import mean_squared_error
import pickle
from algorithms import matching_pursuit, orthogonal_matching_pursuit, weak_orthogonal_matching_pursuit, sparse_LASSO
from concurrent.futures import ThreadPoolExecutor


## 1. Generate Data

In [20]:
def generate_gaussian_noises_dict(N, d, seed=0):
    np.random.seed(seed)
    gaussian_noises = np.random.normal(size=(d, N))
    norms = np.linalg.norm(gaussian_noises, axis=0, keepdims=True)
    # Create unit-norm vectors
    unit_vectors = gaussian_noises / norms
    return unit_vectors

def generate_sparse_response(gaussian_matrix, m, seed=0):
    np.random.seed(seed)
    indices = np.random.choice(gaussian_matrix.shape[1], size=m, replace=False)
    selected_vectors = gaussian_matrix[:, indices]
    coefficients = np.random.normal(size=(m, 1))  # random coefficients for each selected vector
    y = selected_vectors @ coefficients
    return y, indices, coefficients

def generate_perturbed_response(y, noise_level, seed=0):
    np.random.seed(seed)
    norm_y = np.linalg.norm(y)
    noise = np.random.normal(size=y.shape, scale=norm_y * noise_level)
    y_perturbed = y + noise
    return y_perturbed

def generate_perturbed_responses(y, noise_levels, seed=0):
    return [generate_perturbed_response(y, noise_level, seed) for noise_level in noise_levels]

In [21]:
# Use the functions to generate a Gaussian noise matrix and a sparse response
np.random.seed(0)
N = 100000
d = 300
m = 2
dictionary = generate_gaussian_noises_dict(N, d)
y, indices, coefficients = generate_sparse_response(dictionary, m)
noise_scale = 0.05
y_perturbed = generate_perturbed_response(y, noise_scale)

Check coefs

In [22]:
coefficients

array([[1.87413779],
       [0.630206  ]])

Check indices

In [23]:
indices

array([ 3582, 60498])

## Algorithms

### MP

In [16]:
MP_residual, MP_indices, MP_coefficients = matching_pursuit(y, dictionary, 2*m)

MP_residual_perturb, MP_indices_perturb, MP_coefficients_perturb = matching_pursuit(y_perturbed, dictionary, 2*m)

### OMP

In [17]:
OMP_residual, OMP_indices, OMP_coefficients = orthogonal_matching_pursuit(y, dictionary, 2*m)

OMP_residual_perturb, OMP_indices_perturb, OMP_coefficients_perturb = orthogonal_matching_pursuit(y_perturbed, dictionary, 2*m)

### WOMP

In [10]:
WOMP_residual, WOMP_indices, WOMP_coefficients = weak_orthogonal_matching_pursuit(y, dictionary, 0.001)

WOMP_residual_perturb, WOMP_indices_perturb, WOMP_coefficients_perturb = weak_orthogonal_matching_pursuit(y_perturbed, dictionary, 0.001)

## Performance

In [5]:
tested_algorithms = [matching_pursuit, orthogonal_matching_pursuit]

trials = 1 # number of trials

noise_levels = [0, 0.01, 0.05, 0.1]
N = 100000
d = 300
true_sparsity_list = [2, 5, 10, 100, 1000]


final_results = pd.DataFrame()

for true_sparsity in true_sparsity_list:
    m = true_sparsity # True sparsity
    K = m # Model sparsity
    for trial in range(trials):
        # Generate a dictionary and a sparse response
        dictionary = generate_gaussian_noises_dict(N, d, seed=trial)
        y, indices, coefficients = generate_sparse_response(dictionary, m, seed=trial)
        # Generate perturbed responses
        y_perturbeds = generate_perturbed_responses(y, noise_levels, seed=trial)
        sub_final_results = pd.DataFrame()
        # Run algorithms
        for algorithm_index in range(len(tested_algorithms)):
            for noise_level, y_perturbed in zip(noise_levels, y_perturbeds):
                esitmated_residual, estimated_indices, estimated_coefficients = tested_algorithms[algorithm_index](y_perturbed, dictionary, K)
                results = pd.DataFrame({'m': m, 'K': K,'trial': trial, 'algorithm': algorithm_index, 'noise_level': noise_level, 'estimated_indices': [estimated_indices], 'estimated_coefficients': [estimated_coefficients]})
                final_results = pd.concat([final_results, results], ignore_index=True)
        sub_final_results['trial'] = trial
        sub_final_results['noise_level'] = noise_level
        sub_final_results['indices'] = [indices]
        sub_final_results['coefficients'] = [coefficients]
    sub_final_results['m'] = m
    sub_final_results['K'] = K
    final_results = pd.concat([final_results, sub_final_results], ignore_index=True)

LinAlgError: Singular matrix

In [ ]:
final_results

In [24]:
def run_trial(params):
    true_sparsity, trial, noise_levels, tested_algorithms, N, d = params
    m = true_sparsity # True sparsity
    K = m # Model sparsity
    final_results = pd.DataFrame()

    # Generate a dictionary and a sparse response
    dictionary = generate_gaussian_noises_dict(N, d, seed=trial)
    y, indices, coefficients = generate_sparse_response(dictionary, m, seed=trial)
    # Generate perturbed responses
    y_perturbeds = generate_perturbed_responses(y, noise_levels, seed=trial)
    # Run algorithms
    for algorithm_index in range(len(tested_algorithms)):
        for noise_level, y_perturbed in zip(noise_levels, y_perturbeds):
            estimated_residual, estimated_indices, estimated_coefficients = tested_algorithms[algorithm_index](y_perturbed, dictionary, K)
            results = pd.DataFrame({'m': m, 'K': K,'trial': trial, 'algorithm': algorithm_index, 'noise_level': noise_level, 'estimated_indices': [estimated_indices], 'estimated_coefficients': [estimated_coefficients]})
            final_results = pd.concat([final_results, results], ignore_index=True)

    return final_results

tested_algorithms = [matching_pursuit, orthogonal_matching_pursuit]
trials = 100 # number of trials
noise_levels = [0, 0.01, 0.05, 0.1]
N = 100000
d = 300
true_sparsity_list = [2, 5, 10, 20, 100, 200, 2000]
params = [(true_sparsity, trial, noise_levels, tested_algorithms, N, d) for true_sparsity in true_sparsity_list for trial in range(trials)]

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(run_trial, params))

final_results = pd.concat(results, ignore_index=True)

KeyboardInterrupt: 

# Lasso

In [31]:
LASSO_residual, LASSO_indices, LASSO_coefficients = sparse_LASSO(y, dictionary, 0.001)

LASSO_residual_perturb, LASSO_indices_perturb, LASSO_coefficients_perturb = sparse_LASSO(y_perturbed, dictionary, 0.001)

In [2]:
first_attempt = pd.read_pickle('results.pkl')